# Data Mining II Practical Assignment

## Iván Marcelo Carrera Izurieta
### Programa de Doutoramento em Ciência de Computadores FCUP.

#### Predicting Cellular line - Drug Interactions as Recommender System
This work presents an analysis for the relationships among cellular lines and pharmaceutical drugs.

There are two main source databases for cellular lines: [Cellosaurus](https://web.expasy.org/cellosaurus/)
and [ChEMBL](https://www.ebi.ac.uk/chembl/).

##### Text processing

PubMed publications about each cellular line have been retrieved. Corpus is formed from abstracts, and mapped to cell line identifiers.

In [1]:
import pandas as pd
cell = pd.read_csv('https://raw.githubusercontent.com/elprofe-ivan/passignment/master/cell_mapping.csv',
                   sep=',', error_bad_lines=False, encoding="latin-1")
cell['cell_id'] = cell['cell_id'].values.astype(str)
print('Cell line table has columns:', cell.columns, '\n and contains information of', cell.shape[0], 'cell lines')

corpus_df = pd.read_csv('https://raw.githubusercontent.com/elprofe-ivan/passignment/master/corpus_df.csv', sep=',', error_bad_lines=False, encoding="latin-1")
print('Corpus contains information from', corpus_df.shape[0], 'cell lines.')
print('='*80)
print(cell[['cell_chembl_id', 'cell_id', 'cell_name',
       'cell_source_organism', 'cell_source_tissue']].head())
print('='*80, '\nCorpus:')
print(corpus_df.head())
print('='*80)

Cell line table has columns: Index(['cell_chembl_id', 'cell_description', 'cell_id', 'cell_name',
       'cell_source_organism', 'cell_source_tax_id', 'cell_source_tissue',
       'cellosaurus_id', 'cl_lincs_id', 'clo_id', 'efo_id'],
      dtype='object') 
 and contains information of 1554 cell lines
Corpus contains information from 807 cell lines.
  cell_chembl_id cell_id cell_name cell_source_organism   cell_source_tissue
0  CHEMBL3307241       1      DC3F   Cricetulus griseus                 Lung
1  CHEMBL3307242       2    P3HR-1         Homo sapiens              Lyphoma
2  CHEMBL3307243       3  UCLA P-3         Homo sapiens  Lung Adenocarcinoma
3  CHEMBL3307244       4  UMSCC22B         Homo sapiens            Carcinoma
4  CHEMBL3307245       5     UMUC3         Homo sapiens    Bladder Carcinoma
Corpus:
   cell_id                                          documents
0        1   Electroporation enhances mitomycin C cytotoxi...
1        5   Dihydroartemisinin suppresses bladder canc

Corpus will characterize cellular lines, and will help measure distances between cellular lines.

In [2]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

stem = PorterStemmer()
stemmed_corpus = list()
for doc in corpus_df['documents']:
    doc = doc.replace('.','').replace(',','').replace(';','').replace(':','')
    stemmed_corpus.append(' '.join([stem.stem(x) for x in doc.split()]))

corpus_df['stemmed'] = stemmed_corpus
print('='*80, '\nStemmed Corpus:')
print(corpus_df[['cell_id', 'stemmed']].head())
print('='*80)
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=1.0, min_df=2, strip_accents='ascii', stop_words='english',
                             token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
dtm = vectorizer.fit_transform(corpus_df['stemmed'])
print(dtm.shape[1], 'different words in corpus')
print('='*80)

wordfreq = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print('Top 20 frequent words:')
print(sorted(wordfreq.items(), key=lambda x: x[1])[:20])

from sklearn.metrics.pairwise import cosine_similarity

dtm_df = pd.DataFrame(dtm.todense())
sim_mtx = pd.DataFrame(cosine_similarity(dtm_df), index=corpus_df.index, columns=corpus_df.index)
print('='*80)
print('Cell Similarity matrix:', sim_mtx.shape)

Stemmed Corpus:
   cell_id                                            stemmed
0        1  electropor enhanc mitomycin C cytotox on t24 b...
1        5  dihydroartemisinin suppress bladder cancer cel...
2       13  quantif of elastase-lik activ in 13 human canc...
3       21  correct to a1cf-axin2 signal axi regul apoptos...
4       27  cytotox effect of fascaplysin against small ce...
26080 different words in corpus
Top 20 frequent words:
[('cell', 1.0012383902511466), ('line', 1.002478316014467), ('wa', 1.0429901849313683), ('use', 1.0798364583173345), ('studi', 1.0879118723228798), ('result', 1.0960530299065798), ('express', 1.1001485987713169), ('thi', 1.1015175243786586), ('effect', 1.1139229212361463), ('human', 1.1180924894728712), ('activ', 1.137783702363053), ('cancer', 1.1392051667104304), ('tumor', 1.1932723879807061), ('increas', 1.2008082186694091), ('protein', 1.2008082186694091), ('treatment', 1.2008082186694091), ('inhibit', 1.206878040036485), ('induc', 1.20992682288584

Cell line similarity will be helpful when looking for new interactions.

##### Collaborative-Filtering Recommender System

Cell line - Compound interactions are recorded in ChEMBL database.
However, not every cell line - compound combination is annotated in the database.
Using matrix decomposition, values can be computed.

In [3]:
import pandas as pd

# Compounds and cell lines are described in CSV
compound = pd.read_csv('https://raw.githubusercontent.com/elprofe-ivan/passignment/master/comp_recsys.csv', sep=',', error_bad_lines=False, encoding="latin-1")
print('Compound table has columns:', compound.columns)
print(compound.head())
print('\nfp represents Morgan Fingerprint, an expression used in Cheminformatics to represent chemical compounds as a bit string')
print('='*80)
# Table activity contains the relations between compounds and cells
# When a chemical compound has a reported activity on a cell line, 'active' column is set to 1,
# and when a chemical compound has a reported inactivity on a cell line, 'active' is set to 0
activity = pd.read_csv('https://raw.githubusercontent.com/elprofe-ivan/passignment/master/activity_recsys.csv', sep=',', error_bad_lines=False, encoding="latin-1")
activity['cell_id'] = activity['cell_id'].values.astype(str)
# print(activity.head())

cell_activity = pd.merge(activity, cell, on='cell_id')
cols = ['cell_description', 'cell_source_organism', 'cell_source_tax_id', 'cell_source_tissue', 'cl_lincs_id', 'clo_id', 'efo_id']
cell_activity.drop(cols, axis=1, inplace=True)

activity_count = (cell_activity.
     groupby(by = ['cell_name'])['comp_id'].
     count().
     reset_index().
     rename(columns = {'comp_id': 'comp_count'})
     [['cell_name', 'comp_count']]
    )

# Filter cells that report less than 10 activities
threshold = 10
activity_count = activity_count.query('comp_count >= @threshold')
print('Cell lines report activities with between', activity_count['comp_count'].max(), 'and', 
      activity_count['comp_count'].min(), 'compounds')

compound_activity = pd.merge(activity_count, cell_activity, left_on='cell_name', right_on='cell_name', how='left')
compound_count = (compound_activity.
     groupby(by = ['comp_id'])['active'].
     count().
     reset_index().
     rename(columns = {'active': 'activitycount_comp'})
     [['comp_id', 'activitycount_comp']]
    )

# Filter compounds that report less than 10 activities
threshold = 10
compound_count = compound_count.query('activitycount_comp >= @threshold')
print('Compounds report activities with between', compound_count['activitycount_comp'].max(), 'and',
      compound_count['activitycount_comp'].min(), 'cell lines')

combined = compound_activity.merge(compound_count, left_on = 'comp_id', right_on = 'comp_id', how = 'inner')

print('Number of cells: ', combined['cell_name'].nunique())
print('Number of compounds: ', combined['comp_id'].nunique())

combined['active'] = combined['active'].values.astype(float)
combined['cell_id'] = combined['cell_id'].values.astype(str)
combined['comp_id'] = combined['comp_id'].values.astype(str)

combined = combined.drop_duplicates(['comp_id', 'cell_id'])
compound_cell_matrix = combined.pivot(index='comp_id', columns='cell_id', values='active')
# inactivities are considered as -1
compound_cell_matrix = compound_cell_matrix.replace(to_replace=0.0, value=-1.0)
compound_cell_matrix.fillna(0, inplace=True)

compounds = compound_cell_matrix.index.tolist()
cells = compound_cell_matrix.columns.tolist()

Compound table has columns: Index(['id', 'smile', 'fp'], dtype='object')
   id                                              smile  \
0   0  COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H]5C...   
1   1  CN1C2N(CCc3c2n(CCCCCCOc4no[n+]([O-])c4S(=O)(=O...   
2   2  CCOc1ccc(Cl)cc1c2cc(Nc3cccc(c3)[N+](=O)[O-])nc...   
3   3   O=C1N[C@@H](Cc2c[nH]c3ccccc23)C(=O)N4CCC[C@@H]14   
4   4          CCc1ccccc1NS(=O)(=O)c2ccc(NC(=O)NCCCl)cc2   

                                                  fp  
0  1101000000000011000001010000001011101000011100...  
1  0000100000000110000000000001101001000000000001...  
2  0000000000000001000000000000000001000000000001...  
3  0000110000100000000100000000101000000000000000...  
4  0000100000000000000101000000000001000000000000...  

fp represents Morgan Fingerprint, an expression used in Cheminformatics to represent chemical compounds as a bit string
Cell lines report activities with between 21853 and 10 compounds
Compounds report activities with between 664 and 1

For computing matrix decomposition, TensorFlow package is used.

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

num_input = combined['cell_id'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

# top_recomm represent how many top recommendations do you want to store
top_recomm = 10

import numpy as np

with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(compound_cell_matrix.shape[0] / batch_size)
    compound_cell_matrix = np.array_split(compound_cell_matrix, num_batches)

    for i in range(epochs):

        avg_cost = 0
        for batch in compound_cell_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

#         print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    compound_cell_matrix = np.concatenate(compound_cell_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: compound_cell_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='active')
    pred_data.columns = ['comp_id', 'cell_id', 'active']
    pred_data['comp_id'] = pred_data['comp_id'].map(lambda value: compounds[value])
    pred_data['cell_id'] = pred_data['cell_id'].map(lambda value: cells[value])

    keys = ['comp_id', 'cell_id']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    recomm = pred_data[~index_1.isin(index_2)]
    recomm = recomm.sort_values(['comp_id', 'active'], ascending=[True, False])
    # Comment next line for retrieving all recommendations
    recomm = recomm.groupby('comp_id').head(top_recomm)
   

recomm['comp_id'] = recomm['comp_id'].values.astype(int) # for compatibility
print('Recommendations computed')

Recommendations computed


A sample of recommendations can be shown:

In [5]:
sample = recomm['comp_id'].sample(2)
print(sample)
print('='*80)
for comp_id in sample:
    print(recomm[recomm['comp_id']==comp_id])

272393     13010
198685    122469
Name: comp_id, dtype: int64
        comp_id cell_id    active
272352    13010     721  0.469569
272239    13010     555  0.416721
272310    13010     655  0.389963
272109    13010     325  0.346262
272294    13010     635  0.252373
272333    13010     691  0.230858
272188    13010     478  0.202418
272393    13010     791  0.187265
272220    13010     525  0.142051
272374    13010     757  0.138680
        comp_id cell_id    active
198768   122469     721  0.523102
198728   122469     657  0.486062
198788   122469     755  0.466567
198658   122469     560  0.344890
198672   122469     577  0.335617
198525   122469     325  0.319936
198777   122469     743  0.301083
198685   122469     600  0.296588
198786   122469     753  0.281515
198751   122469     693  0.272080


##### Combining Recommendations

For a cold-start recommendation, let's imagine a novel compound which has no reported activities.

In [25]:
import numpy as np
from sklearn.metrics import jaccard_score


def fp_distance(elem_):
    i, j = elem_[0], elem_[1]
    fp_i = np.array(list(compound.iloc[i]['fp']), dtype=int)
    fp_j = np.array(list(compound.iloc[j]['fp']), dtype=int)
    s = jaccard_score(y_true=fp_i, y_pred=fp_j)
    return s


def distance_list(i, cutoff=0.3):
    # print('Computing distance with', n_comp,'compounds.')
    n_comp = combined['comp_id'].nunique()
    iter_ = zip([i]*n_comp, range(n_comp))
    # This should be done in parallel
    # Binder does not support multiprocessing package
    dist_list = list()
    for item_ in iter_:
        dist_list.append(fp_distance(item_))
    dist_list = zip(compound['id'], dist_list)
    # Return only compounds that have a similarity higher than cutoff
    dist_list = list(filter(lambda elem_: elem_[1] > cutoff, dist_list))
    return sorted(dist_list, key=lambda x: x[1], reverse=True)


def cell_distance_list(cell_id, cutoff=0.3):
    cell_list = sim_mtx[cell_id][sim_mtx[cell_id] > cutoff]
    return cell_list.drop(cell_id)


def get_cells_for_comp(comp_id):
    return activity[activity['comp_id'] == int(comp_id)]


def get_comps_for_cell(cell_id):
    return activity[activity['cell_id'] == cell_id]


def cold_start_recomm(example_comp_id):
    s = recomm.loc[recomm['comp_id'] == example_comp_id]
    candidate_cells = set()
    if s.empty:
        print(example_comp_id, 'not in Collaborative-Filtering Recommendations')
        reported_cells = get_cells_for_comp(example_comp_id)['cell_id'].values
        candidate_cells.update(reported_cells)
        print('Reported cellular lines for', example_comp_id,' :', reported_cells)
        for cell in reported_cells:
            similar_cells = list(cell_distance_list(int(cell)).index)
            candidate_cells.update(similar_cells)
            print('Similar cells to', cell,':', similar_cells)
            
        dist_list = distance_list(example_comp_id)
        if len(dist_list) > 1:
            print('Compound id', example_comp_id, 'has', len(dist_list)-1, 'similar compounds:',
                  [dist_item[0] for dist_item in dist_list[1:]])
            for comp in dist_list[1:]:
                s = recomm.loc[recomm['comp_id'] == comp[0]]
                if s.empty:
                    print(comp[0], 'not in Collaborative-Filtering Recommendations')
                    reported_cells = get_cells_for_comp(comp[0])['cell_id'].values
                    candidate_cells.update(reported_cells)
                    print('Reported cellular lines for', comp[0],' :', reported_cells)
                    for cell in reported_cells:
                        similar_cells = list(cell_distance_list(int(cell)).index)
                        candidate_cells.update(similar_cells)
                        print('Similar cells to', cell,':', similar_cells)
                else:
                    print('Collaborative-Filtering Recommendations for: ', comp[0])
                    print(s['cell_id'].values)
        else:
            print('Compound id', example_comp_id, 'has no similar compounds')
        return candidate_cells
    else:
        print('Collaborative-Filtering Recommendations for: ', example_comp_id)
        print(s['cell_id'].values)
        return s['cell_id'].values
    
            
            

example_comp_id = 1
print('Candidate cells for', example_comp_id, ':',cold_start_recomm(example_comp_id))
print('='*80)


1 not in Collaborative-Filtering Recommendations
Reported cellular lines for 1  : ['646']
Similar cells to 646 : []
Compound id 1 has 2 similar compounds: [1022, 550]
1022 not in Collaborative-Filtering Recommendations
Reported cellular lines for 1022  : ['646' '687' '721']
Similar cells to 646 : []
Similar cells to 687 : []
Similar cells to 721 : [773]
550 not in Collaborative-Filtering Recommendations
Reported cellular lines for 550  : ['436' '726' '791']
Similar cells to 436 : []
Similar cells to 726 : [20, 671]
Similar cells to 791 : [521]
Candidate cells for 1 : {'721', 773, 521, '687', '646', '436', 20, '726', '791', 671}
